In [1]:
import pandas as pd

In [3]:
!pip install pandas joblib lightgbm catboost xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.4 MB/s eta 0:00:00


In [5]:
import joblib

# Load the test data
test_data = pd.read_csv("/content/drive/MyDrive/NASA Hackathon/Final Test Datasets Scripts/5year.csv")

# Extracting the date for final CSV
dates = test_data['Date']

# Removing the 'Date' and 'Time' columns for predictions
test_data = test_data.drop(columns=['Date', 'Time'])

def hour_ensemble_predictions(input_data):
    # Load models
    lightgbm_model = joblib.load("/content/drive/MyDrive/NASA Hackathon/Final Test Datasets Scripts/lgbm_model.pkl")
    catboost_model = joblib.load("/content/drive/MyDrive/NASA Hackathon/Final Test Datasets Scripts/catboost_model.pkl")
    xgboost_model = joblib.load("/content/drive/MyDrive/NASA Hackathon/Final Test Datasets Scripts/xgboost_model.pkl")

    # Weights based on R^2 scores
    weights = {
        "LightGBM": 0.9011,
        "CatBoost":  0.8874,
        "XGBoost": 0.8972
    }

    # Normalize the weights to make sure they sum to 1
    total_weight = sum(weights.values())
    for model in weights:
        weights[model] /= total_weight

    # Predict using individual models
    lgbm_pred = lightgbm_model.predict(input_data)
    cat_pred = catboost_model.predict(input_data)
    xgb_pred = xgboost_model.predict(input_data)

    # Compute the weighted average of predictions
    final_prediction = (lgbm_pred * weights["LightGBM"] +
                        cat_pred * weights["CatBoost"] +
                        xgb_pred * weights["XGBoost"])

    return final_prediction

# Generate predictions using the ensemble method
predictions = hour_ensemble_predictions(test_data)

# Create the final CSV with Date and Predicted kp values
final_predictions_df = pd.DataFrame({
    'Date': dates,
    'Predicted_KP': predictions
})

# Save the final predictions to a CSV file
prediction_file_path = "//content/drive/MyDrive/NASA Hackathon/Final Test Datasets Scripts/final_predictions.csv"
final_predictions_df.to_csv(prediction_file_path, index=False)

prediction_file_path

'//content/drive/MyDrive/NASA Hackathon/Final Test Datasets Scripts/final_predictions.csv'